In [11]:
def fcfs(processes):
    processes = sorted(processes, key=lambda x: x.arrival_time)
    current_time = 0

    for process in processes:
        if current_time < process.arrival_time:
            current_time = process.arrival_time
        process.waiting_time = current_time - process.arrival_time
        process.turnaround_time = process.waiting_time + process.burst_time
        current_time += process.burst_time

    avg_waiting_time = sum([p.waiting_time for p in processes]) / len(processes)
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / len(processes)

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [12]:
def round_robin(processes, quantum):
    time = 0
    ready_queue = []
    completed_processes = 0
    n = len(processes)

    while completed_processes < n:
        # Adicione processos que chegaram ao tempo atual na fila de pronto
        for p in processes:
            if p.arrival_time == time and p.remaining_time > 0:
                ready_queue.append(p)

        # Execute o processo na frente da fila de pronto
        if ready_queue:
            current_process = ready_queue.pop(0)

            if current_process.remaining_time > quantum:
                time += quantum
                current_process.remaining_time -= quantum

                # Adicione novamente à fila se não estiver concluído
                for p in processes:
                    if p.arrival_time <= time and p.remaining_time > 0 and p not in ready_queue:
                        ready_queue.append(p)

                if current_process.remaining_time > 0:
                    ready_queue.append(current_process)
            else:
                time += current_process.remaining_time
                current_process.turnaround_time = time - current_process.arrival_time
                current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
                current_process.remaining_time = 0
                completed_processes += 1
        else:
            time += 1

    avg_waiting_time = sum([p.waiting_time for p in processes]) / len(processes)
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / len(processes)

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [13]:
def sjf(processes):
    n = len(processes)
    completed = 0
    current_time = 0
    processes = sorted(processes, key=lambda x: (x.arrival_time, x.burst_time))

    while completed < n:
        available_processes = [p for p in processes if p.arrival_time <= current_time and p.turnaround_time == 0]

        # Se não houver processos disponíveis, apenas avance o tempo
        if not available_processes:
            current_time += 1
            continue

        # Escolha o processo com o menor burst time
        next_process = min(available_processes, key=lambda x: x.burst_time)
        current_time += next_process.burst_time
        next_process.turnaround_time = current_time - next_process.arrival_time
        next_process.waiting_time = next_process.turnaround_time - next_process.burst_time
        completed += 1

    avg_waiting_time = sum([p.waiting_time for p in processes]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / n

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [14]:
def srtf(processes):
    current_time = 0
    completed_processes = 0
    n = len(processes)
    current_process = None

    while completed_processes < n:
        # Encontrar os processos que já chegaram
        available_processes = [p for p in processes if p.arrival_time <= current_time and not p.completed]

        # Se não houver processos disponíveis, avance o tempo
        if not available_processes:
            current_time += 1
            continue

        # Selecione o processo com o menor tempo restante
        shortest_process = min(available_processes, key=lambda x: x.remaining_time)

        # Se não há processo atual ou se o processo atual tem um tempo restante maior que o novo processo
        if not current_process or current_process.remaining_time > shortest_process.remaining_time:
            current_process = shortest_process

        # Decrementa o tempo restante do processo atual
        current_process.remaining_time -= 1
        current_time += 1

        # Verificar se o processo atual foi concluído
        if current_process.remaining_time == 0:
            current_process.completed = True
            current_process.turnaround_time = current_time - current_process.arrival_time
            current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
            completed_processes += 1
            current_process = None

    avg_waiting_time = sum([p.waiting_time for p in processes]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / n

    print(f"{'Processo':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [15]:
def priority_non_preemptive(processes):
    current_time = 0
    completed_processes = 0
    n = len(processes)
    processes = sorted(processes, key=lambda x: x.arrival_time)  # Ordenar por tempo de chegada inicialmente

    while completed_processes < n:
        # Encontrar os processos que já chegaram
        available_processes = [p for p in processes if p.arrival_time <= current_time and p.turnaround_time == 0]

        # Se não houver processos disponíveis, avance o tempo
        if not available_processes:
            current_time += 1
            continue

        # Selecione o processo com a prioridade mais alta (valor mais baixo indica alta prioridade)
        highest_priority_process = min(available_processes, key=lambda x: x.priority)
        current_time += highest_priority_process.burst_time
        highest_priority_process.turnaround_time = current_time - highest_priority_process.arrival_time
        highest_priority_process.waiting_time = highest_priority_process.turnaround_time - highest_priority_process.burst_time
        completed_processes += 1

    avg_waiting_time = sum([p.waiting_time for p in processes]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / n

    print(f"{'Processo':<10} {'Prioridade':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.priority:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [16]:
def priority_preemptive(processes):
    current_time = 0
    completed_processes = 0
    n = len(processes)
    current_process = None

    while completed_processes < n:
        # Encontrar os processos que já chegaram
        available_processes = [p for p in processes if p.arrival_time <= current_time and not p.completed]

        # Se não houver processos disponíveis, avance o tempo
        if not available_processes:
            current_time += 1
            continue

        # Selecione o processo com a prioridade mais alta (valor mais baixo indica alta prioridade)
        highest_priority_process = min(available_processes, key=lambda x: (x.priority, x.arrival_time))

        # Se há um processo em execução e a sua prioridade é menor que o processo atualmente em execução
        if current_process and highest_priority_process.priority < current_process.priority:
            current_process = highest_priority_process

        # Se não há processo em execução, selecione o processo de maior prioridade
        if not current_process:
            current_process = highest_priority_process

        # Se o processo atual não foi executado antes ou foi interrompido
        if current_process.last_executed_at != current_time - 1:
            current_process.waiting_time += current_time - max(current_process.last_executed_at, current_process.arrival_time)

        current_process.last_executed_at = current_time
        current_process.remaining_time -= 1
        current_time += 1

        # Verificar se o processo atual foi concluído
        if current_process.remaining_time == 0:
            current_process.completed = True
            current_process.turnaround_time = current_time - current_process.arrival_time
            completed_processes += 1
            current_process = None

    avg_waiting_time = sum([p.waiting_time for p in processes]) / n
    avg_turnaround_time = sum([p.turnaround_time for p in processes]) / n

    print(f"{'Processo':<10} {'Prioridade':<10} {'Tempo de Espera':<15} {'Tempo de Execução':<15}")
    for p in processes:
        print(f"{p.name:<10} {p.priority:<10} {p.waiting_time:<15} {p.turnaround_time:<15}")

    print(f"\nTempo de Espera Médio: {avg_waiting_time:}")
    print(f"Tempo de Execução Médio: {avg_turnaround_time:}")


In [17]:
class Process:
    def __init__(self, name, arrival_time, burst_time, priority=None):
        self.name = name
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.priority = priority
        self.waiting_time = 0
        self.turnaround_time = 0
        self.completed = False
        self.last_executed_at = -1

#def fcfs(processes):
    # ... (código FCFS aqui) ...

#def round_robin(processes, quantum):
    # ... (código Round Robin aqui) ...

#def sjf(processes):
    # ... (código SJF aqui) ...

#def srtf(processes):
    # ... (código SRTF aqui) ...

#def priority_non_preemptive(processes):
    # ... (código Priority Non-Preemptive aqui) ...

#def priority_preemptive(processes):
    # ... (código Priority Preemptive aqui) ...

if __name__ == '__main__':
    n = int(input("Digite o número de processos: "))
    processes = []

    for i in range(n):
        name = input(f"\nNome do processo {i+1}: ")
        arrival_time = int(input(f"Tempo de chegada do processo {name}: "))
        burst_time = int(input(f"Tempo de execução do processo {name}: "))
        priority = None
        has_priority = input(f"O processo {name} tem uma prioridade? (s/n) ").lower().strip()
        if has_priority == 's':
            priority = int(input(f"Prioridade do processo {name} (número menor indica maior prioridade): "))
        processes.append(Process(name, arrival_time, burst_time, priority))

    print("\nEscolha o algoritmo de escalonamento:")
    print("1. FCFS")
    print("2. Round Robin")
    print("3. SJF")
    print("4. SRTF")
    print("5. Prioridade Não Preemptiva")
    print("6. Prioridade Preemptiva")

    choice = int(input("\nDigite o número correspondente ao algoritmo: "))

    if choice == 1:
        fcfs(processes)
    elif choice == 2:
        quantum = int(input("\nDigite o quantum para o Round Robin: "))
        round_robin(processes, quantum)
    elif choice == 3:
        sjf(processes)
    elif choice == 4:
        srtf(processes)
    elif choice == 5:
        priority_non_preemptive(processes)
    elif choice == 6:
        priority_preemptive(processes)
    else:
        print("Escolha inválida.")

Digite o número de processos: 2

Nome do processo 1: P1
Tempo de chegada do processo P1: 0
Tempo de execução do processo P1: 3
O processo P1 tem uma prioridade? (s/n) n

Nome do processo 2: P2
Tempo de chegada do processo P2: 1
Tempo de execução do processo P2: 5
O processo P2 tem uma prioridade? (s/n) n

Escolha o algoritmo de escalonamento:
1. FCFS
2. Round Robin
3. SJF
4. SRTF
5. Prioridade Não Preemptiva
6. Prioridade Preemptiva

Digite o número correspondente ao algoritmo: 1
Processo   Tempo de Espera Tempo de Execução
P1         0               3              
P2         2               7              

Tempo de Espera Médio: 1.0
Tempo de Execução Médio: 5.0
